In [2]:
import sys

sys.path.append('../../..')

1) Prepare the dataset in the pytorch Dataset format.
For example like this with COCO annotation format:

In [ ]:
from torchvision.transforms import ToTensor

from core.architecture.datasets.object_detection_datasets import COCODataset

train_dataset_path = '' # your path to train part of dataset
val_dataset_path = '' # your path to validation part of dataset
train_json_path = '' # your path to train annotations of dataset
val_json_path = '' # your path to validation annotations of dataset

transform = ToTensor()
train_dataset = COCODataset(images_path=train_dataset_path, json_path=train_json_path, transform=transform)
val_dataset = COCODataset(images_path=val_dataset_path, json_path=val_json_path, transform=transform)

2) Model initialization.

In [4]:
from core.architecture.experiment.nn_experimenter import FasterRCNNExperimenter

experimenter = FasterRCNNExperimenter(
    num_classes = len(train_dataset.classes) + 1,
    model_params={'weights': 'DEFAULT'}
)

3) Fit parameters.

In [5]:
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from core.architecture.experiment.nn_experimenter import FitParameters

dl_params = {
    'batch_size': 2,
    'num_workers': 2,
    'collate_fn': lambda x: tuple(zip(*x))
}

fit_params = FitParameters(
    dataset_name='ALET',
    train_dl=DataLoader(train_dataset, shuffle=True, **dl_params),
    val_dl=DataLoader(val_dataset, **dl_params),
    num_epochs=50,
    optimizer_params={'lr': 0.0001},
    lr_scheduler=StepLR,
    lr_scheduler_params={'step_size': 10}
)

4) Initialization of the structure optimization (optionally).

In [6]:
from core.operation.optimization.structure_optimization import SVDOptimization

svd_optimization = SVDOptimization(energy_thresholds=[0.1, 0.3, 0.5, 0.7, 0.9, 0.99, 0.995, 0.999, 1])

5) Running Model Training.
If you want to use structure optimization use ``svd_optimization.fit()`` instead of ``experimenter.fit()``

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

experimenter.fit(p=fit_params)
# svd_optimization.fit(exp=experimenter, params=fit_params)

Now you can get predictions for all images in a folder. The method returns a dictionary {'image name': predictions}

In [ ]:
from core.architecture.datasets.prediction_datasets import PredictionFolderDataset

prediction_dataset = PredictionFolderDataset(
    image_folder='', # Path to your folder with images
    transform=transform,
)
prediction_dl = DataLoader(prediction_dataset, **dl_params)
preds = experimenter.predict(prediction_dl)
preds